In [6]:
%load_ext autoreload
%autoreload 2

import torch
import yaml
import sys
from torchmetrics import MetricCollection
from omegaconf import OmegaConf
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from pytorch_lightning import seed_everything
sys.path.append('fbsource/fbcode/scripts/psarlin/')
from maploc.data.loader_mapillary import MapillaryDataModule
from maploc.module import GenericModule, find_best_checkpoint
from maploc.utils.viz_2d import plot_images, plot_keypoints, features_to_RGB, save_plot, add_text
from maploc.utils.viz_localization import likelihood_overlay, plot_pose, plot_dense_rotations
from maploc.osm.viz import Colormap, plot_nodes
from maploc.models.bev_localizer import PolarProjection
from maploc.models.metrics import Location2DError, AngleError
from maploc.models.hough_voting import argmax_xyr, fuse_gps
from maploc.models.refinement import FeaturemetricRefiner, subpixel_refinement
torch.set_grad_enabled(False);
plt.rcParams.update({'figure.max_open_warning': 0})

In [25]:
conf = OmegaConf.load('fbsource/fbcode/scripts/psarlin/maploc/conf/data_mapillary.yaml')
conf = OmegaConf.merge(conf, OmegaConf.create(yaml.full_load("""
local_dir: "./data/mapillary_dumps_v2/"
# local_dir: "./devvm1417.cln0.facebook.com/data/mapillary_dumps_v2/"
dump_dir: ${.local_dir}
scenes:
    - sanfrancisco_soma
    - sanfrancisco_hayes
    - amsterdam
    - berlin
    - lemans
    - montrouge
    - toulouse
    - nantes
    - vilnius
    - avignon
    - helsinki
    - milan
    # - newyork_hoboken
    # - metropolis_train
# split: None

return_gps: true
val: {batch_size: 1, num_workers: 1}
train: ${.val}
random: false
augmentation: {rot90: false, flip: false}
""")))
OmegaConf.resolve(conf)
dataset = MapillaryDataModule(conf)
dataset.prepare_data()
dataset.setup()

In [24]:
exper = "bev1-osm2-mly12-n100_vgg16-vgg13_bs9-resize256_norm-d8-nrot64"
# exper = "bev1-osm2-mly12-n100_vgg16-vgg13_bs10-resize256_attn-fix-2-2-128d_norm-d8-nrot64"
# exper = "bev1-osm2-mly12-n100_vgg16-vgg13_bs6-resize256_attn-fix-2-2-256d_norm-d8-nrot32"
exper = "bev1-osm2-mly12-n100_vgg16-vgg13-plane_bs9-resize256_norm-d8-nrot64"
# exper = "bev1-osm2-mly12-n100_vgg16-vgg13-plane_bs9-resize256_norm-d8-nrot64-prior"

root = "manifold://psarlin/tree/maploc/experiments"
path = f'{root}/{exper}/last.ckpt'
# path = find_best_checkpoint(f'{root}/{exper}')
print(path)
cfg = {}
cfg = {'model': {"num_rotations": 256}}
model = GenericModule.load_from_checkpoint(path, strict=True, find_best=True, cfg=cfg)
model = model.eval()#.cuda()
proj_polar = PolarProjection(model.cfg.model.z_max, model.cfg.model.pixel_per_meter)

In [25]:
seed_everything(23) # was 17
loader = dataset.dataloader("val", shuffle=True)
print(exper)
colormap = Colormap()
metrics = MetricCollection(model.model.metrics())
metrics["xy_gps"] = Location2DError("xy_gps", model.cfg.model.pixel_per_meter)
metrics["xy_fused"] = Location2DError("xy_fused", model.cfg.model.pixel_per_meter)
# metrics["xy_refined"] = Location2DError("xy_refined", model.cfg.model.pixel_per_meter)
# metrics["xy_subpix"] = Location2DError("xy_subpix", model.cfg.model.pixel_per_meter)
metrics["yaw_fused"] = AngleError("yaw_fused")
# metrics["yaw_refined"] = AngleError("yaw_refined")
# metrics["yaw_subpix"] = AngleError("yaw_subpix")
metrics = metrics.to(model.device)
refiner = FeaturemetricRefiner(model.model.projection_bev.grid_xz, ppm=model.cfg.model.pixel_per_meter)

for i, batch in zip(range(30), loader):
    batch = model.transfer_batch_to_device(batch, model.device, i)
    pred = model(batch)
    uv_gps, = pred["xy_gps"] = batch["xy_gps"]
    uvt_fused = argmax_xyr(fuse_gps(pred["log_probs"], uv_gps, model.cfg.model.pixel_per_meter, sigma=10))
    uv_fused, = pred["xy_fused"] = uvt_fused[..., :2]
    yaw_fused, = pred["yaw_fused"] = uvt_fused[..., -1]
    # pred["xy_refined"], pred["yaw_refined"] = refiner(
    #     uv_fused, yaw_fused, pred["map"]["map_features"][0][0], pred["features_bev"][0],
    #     pred["bev"]["confidence"][0], pred["valid_bev"][0])
    # pred["xy_subpix"], pred["yaw_subpix"] = subpixel_refinement(
    #     pred["log_probs"][0], uv_fused, yaw_fused)

    loss = model.model.loss(pred, batch)
    results = metrics(pred, batch)
    results.pop("xy_expectation_error")
    print(f'{i} {loss["total"].item():.2f}', {k: round(v.item(), 2) for k, v in results.items()})

    scene = batch["scene"][0]
    name = batch["name"][0]
    view = dataset.dumps[scene][batch["sequence"][0]]["views"][name]
    image = batch["image"][0].permute(1, 2, 0)
    rasters = batch["map"][0]
    uv = batch["xy"][0]
    yaw = batch["roll_pitch_yaw"][0, -1].numpy()

    lp_uvt = pred["log_probs"][0]
    lp_uv = lp_uvt.max(-1).values
    prob = lp_uv.exp()
    feats_map = pred["map"]["map_features"][0][0]
    feats_q = pred["features_bev"][0]
    mask_bev = pred["valid_bev"][0]
    if "log_prior" in pred["map"]:
        prior = pred["map"]["log_prior"][0][0].sigmoid()
    else:
        prior = None
    norm_q = torch.norm(feats_q, dim=0)
    conf_q = pred["bev"].get("confidence", norm_q[None])[0].clone()
    conf_q.masked_fill_(~mask_bev, np.nan)

    image_bev, _, _ = model.model.projection_bev(
        *model.model.projection_polar(
            batch["image"], batch["ground_plane"],  batch["camera"])[:2], batch["camera"])
            # batch["image"], batch["camera_height"],  batch["camera"])[:2], batch["camera"])
    image_bev = image_bev[0].permute(1, 2, 0).numpy()
    image_bev = np.concatenate([image_bev, mask_bev[..., None]], -1)

    feats_map_rgb, = features_to_RGB(feats_map.numpy())
    feats_q_rgb, = features_to_RGB(feats_q.numpy(), masks=[mask_bev])
    # feats_map_rgb, feats_q_rgb, = features_to_RGB(feats_map.numpy(), feats_q.numpy(), masks=[None, mask_bev])
    feats_q_rgb = np.concatenate([feats_q_rgb, mask_bev[..., None]], -1)

    text1 = f'error: {results["xy_max_error"]:.1f}m/{results["yaw_max_error"]:.1f}°'
    # text2 = f'{results["xy_gps"]:.1f}/{results["xy_fused"]:.1f}/{results["xy_refined"]:.1f}/{results["xy_subpix"]:.1f}m'
    # text2 = f'{results["xy_gps"]:.1f}/{results["xy_fused"]:.1f}/{results["xy_subpix"]:.1f}m'
    text2 = f'GPS/fused: {results["xy_gps"]:.1f}/{results["xy_fused"]:.1f}m'
    map_viz = colormap.apply(rasters)
    plot_images([image, map_viz, lp_uv, likelihood_overlay(prob.numpy(), map_viz), feats_map_rgb],
                titles=[text1, text2, 'loglikelihood', 'likelihood', 'map features'], dpi=75, cmaps='jet')
    colormap.add_colorbar()
    plot_nodes(1, rasters[2])
    plot_pose([1], uv, yaw, s=1/35, c="forestgreen", w=0.015)
    plot_pose([1], uv_fused, yaw_fused, s=1/35, c="blue", w=0.015)
    plot_pose([1], uv_gps, c="red")
    plot_pose([1], pred["xy_max"][0], pred["yaw_max"][0], s=1/35, c="k", w=0.015)
    plot_dense_rotations(2, lp_uvt.exp(), s=1/15)

    plot_images([image_bev, conf_q, feats_q_rgb] + ([] if prior is None else [prior]),
                 titles=["BEV image", "BEV weight", "BEV features"]+([] if prior is None else ['map prior']), dpi=50, cmaps='jet')

# Save everything

In [ ]:
dataset.splits["val"] = [('nantes', 'BIwoM6f-6qeoP0Qj-4-ZIw', '0AMOwBR8MBXjEmtHO636yA_front')]
loader = dataset.dataloader("val", shuffle=True)
colormap = Colormap()
batch = next(iter(loader))
rasters = batch["map"][0]
map_viz = colormap.apply(rasters)
masked = [colormap.apply(torch.cat([torch.zeros_like(rasters[:i]), r[None], torch.zeros_like(rasters[i+1:])])) for i, r in enumerate(rasters)]
plot_images([map_viz]+masked)
plot_nodes(3, rasters[2])
colormap.add_colorbar()

name2 = "_".join([batch["scene"][0], batch["sequence"][0], batch["name"][0]])
dir_ = "./dumps/final_presentation/"
plot_images([map_viz], dpi=200)
plot_nodes(0, rasters[2])
save_plot(dir_+name2+"_map_raster.png")
plot_images([masked[0]], dpi=200)
save_plot(dir_+name2+"_map_raster_areas.png")
plot_images([masked[1]], dpi=200)
save_plot(dir_+name2+"_map_raster_lines.png")
plot_images([masked[2]], dpi=200)
plot_nodes(0, rasters[2])
save_plot(dir_+name2+"_map_raster_nodes.png")
plot_images([masked[2]], dpi=200)
colormap.add_colorbar()
save_plot(dir_+name2+"_map_raster_labels.png")

In [ ]:
name2 = "_".join([scene, batch["sequence"][0], name])

plot_images([image],dpi=200)
# save_plot(dir_+name2+".png")

feats_image, = features_to_RGB(pred["features_image"][0].numpy())
plot_images([feats_image],dpi=200)
# save_plot(dir_+name2+"_imfeats.png")

plot_images([feats_map_rgb],dpi=200)
# save_plot(dir_+name2+"_mapfeats.png")

plot_images([image_bev],dpi=200)
# save_plot(dir_+name2+"_bev.png", transparent=True)

plot_images([feats_q_rgb],dpi=200)
# save_plot(dir_+name2+"_bevfeats.png", transparent=True)

# Basic with rotation (deprecated)

In [ ]:
seed_everything(23) # was 17
loader = dataset.dataloader("train", shuffle=True)
print(exper)
colormap = Colormap()
for i, batch in zip(range(20), loader):
    pred = model(batch)
    loss = model.model.loss(pred, batch)
    metrics = MetricCollection(model.model.metrics())(pred, batch)
    print(i, loss["total"].item(), metrics)

    scene = batch["scene"][0]
    name = batch["name"][0]
    image = batch["image"][0].permute(1, 2, 0)
    rasters = batch["map"][0]
    xy = batch["xy"][0]
    yaw = batch["roll_pitch_yaw"][0, -1].numpy()

    latlong_gps = dataset.dumps[scene][batch["sequence"][0]]["views"][name]["gps_position"][:2]
    xy_gps = dataset.tile_managers[scene].projection.project(np.array(latlong_gps))
    uv_gps = batch["canvas"][0].to_uv(xy_gps)

    xyp = pred["xy_max"][0].numpy().astype(int)
    lp_xyt = pred["log_probs"][0]
    lp_xy = lp_xyt.max(-1).values
    prob = lp_xy.exp()
    feats_map = pred["map"]["map_features"][0][0]
    feats_map_rgb, = features_to_RGB(feats_map.numpy())

    text = f'error: {metrics["xy_max_error"]:.1f}m/{metrics["yaw_max_error"]:.1f}°'
    map_viz = colormap.apply(rasters)
    plot_images([image, map_viz, lp_xy, likelihood_overlay(prob.numpy(), map_viz), feats_map_rgb],
                titles=[text, 'map', 'loglikelihood', 'likelihood', 'map features'], dpi=75, cmaps='jet')
    colormap.add_colorbar()
    plot_nodes(1, rasters[2])
    plot_pose([1], xy, yaw, s=1/35, c="forestgreen", w=0.015)
    plot_pose([1], uv_gps, c="red")
    plot_pose([1], pred["xy_max"][0], pred["yaw_max"][0], s=1/35, c="k", w=0.015)
    plot_dense_rotations(2, lp_xyt.exp(), s=1/15)